In [7]:
!pip install matplotlib

  Using cached matplotlib-3.8.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.8 kB)
  Using cached contourpy-1.2.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.49.0-cp310-cp310-macosx_10_9_universal2.whl.metadata (159 kB)
  Using cached kiwisolver-1.4.5-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.4 kB)
Using cached matplotlib-3.8.3-cp310-cp310-macosx_11_0_arm64.whl (7.5 MB)
Using cached contourpy-1.2.0-cp310-cp310-macosx_11_0_arm64.whl (242 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.49.0-cp310-cp310-macosx_10_9_universal2.whl (2.8 MB)
Using cached kiwisolver-1.4.5-cp310-cp310-macosx_11_0_arm64.whl (66 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 2.1 MB/s eta 0:00:00 0:00:01


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import json
import gensim.downloader as api
from sklearn.metrics import f1_score
import numpy as np
from gensim.models import KeyedVectors
import pickle
import matplotlib.pyplot as plt

In [2]:
with open('Json Task1/train_processed.json', 'r') as f:
    task1_train_data = json.load(f)

with open('Json Task1/val_processed.json', 'r') as f:
    task1_val_data = json.load(f)

with open('Json Task1/test_processed.json', 'r') as f:
    task1_test_data = json.load(f)

with open('Json Task2/train_processed.json', 'r') as f:
    task2_train_data = json.load(f)

with open('Json Task2/val_processed.json', 'r') as f:
    task2_val_data = json.load(f)

with open('Json Task2/test_processed.json', 'r') as f:
    task2_test_data = json.load(f)

with open('glove_embedding.pkl', 'rb') as pickle_file:
    glove_embeddings = pickle.load(pickle_file)
    
with open('fast_text_embedding.pkl', 'rb') as pickle_file:
    fast_text_embedding = pickle.load(pickle_file)

bio_mapping_task1 = {'B_ORG': 0, 'I_ORG': 1, 'B_RESPONDENT': 2, 'I_RESPONDENT': 3, 'B_JUDGE': 4, 'I_JUDGE': 5,
               'B_STATUTE': 6, 'I_STATUTE': 7, 'B_OTHER_PERSON': 8, 'I_OTHER_PERSON': 9, 'B_COURT': 10, 'I_COURT': 11,
               'B_GPE': 12, 'I_GPE': 13, 'B_PETITIONER': 14, 'I_PETITIONER': 15, 'B_WITNESS': 16, 'I_WITNESS': 17,
               'B_CASE_NUMBER': 18, 'I_CASE_NUMBER': 19, 'B_PRECEDENT': 20, 'I_PRECEDENT': 21, 'B_DATE': 22, 'I_DATE': 23,
               'B_PROVISION': 24, 'I_PROVISION': 25, 'O': 26}
bio_mapping_task2 = {'O' : 0, 'I' : 1, 'B' : 2}

In [3]:
word2vec_model = api.load('word2vec-google-news-300')

word2vec = torch.FloatTensor(word2vec_model.vectors)
word2vec_u_ = np.concatenate((word2vec, np.zeros((1, 300), dtype = 'float32')), axis = 0)

# Add an unknown token to the vocabulary
word_to_index = {word: index for index, word in enumerate(word2vec_model.index_to_key)}
word_to_index['<unk>'] = len(word_to_index)

# Example usage
unknown_token_index = word_to_index['<unk>']

In [2]:
# fasttext_model_path = "wiki-news-300d-1M-subword.vec"
# fasttext_model = KeyedVectors.load_word2vec_format(fasttext_model_path, binary=False)

# def get_word_embedding(word):
#     if word in fasttext_model:
#         return fasttext_model[word]
#     else:
#         subword_embeddings = [fasttext_model[subword] for subword in fasttext_model.key_to_index if subword in word]
#         if subword_embeddings:
#             return np.mean(subword_embeddings, axis=0)
#         else:
#             return np.zeros(fasttext_model.vector_size)

# fast_text_embedding = {}
# def get_word_vector():
#     count = 0
#     data = [task1_train_data, task1_val_data, task1_test_data, task2_train_data, task2_val_data, task2_test_data]
#     for task_data in data:
#         for key in task_data:
#             words = task_data[key]['text'].split(' ')
#             print(count)
#             count +=1
#             for word in words:
#                 if word not in fast_text_embedding:
#                     word_vector = get_word_embedding(word)
#                     fast_text_embedding[word] = word_vector


# get_word_vector()  

# def load_glove_embeddings(file_path):
#     word_embeddings = {}
#     with open(file_path, 'r', encoding='utf-8') as file:
#         for line in file:
#             try:
#                 values = line.split()
#                 word = values[0]
#                 vector = np.array(values[1:], dtype='float32')
#                 word_embeddings[word] = vector
#             except ValueError as e:
#                 print(f"Error processing line: {line}\nError: {e}")
#     return word_embeddings

# glove_file_path = 'glove.840B.300d.txt'  # Adjust the path based on your downloaded file
# glove_embeddings = load_glove_embeddings(glove_file_path)

# g_e = {}
# def get_glove_dictionary(glove_embeddings):
#     count = 0
#     data = [task1_train_data, task1_val_data, task1_test_data, task2_train_data, task2_val_data, task2_test_data]
#     for task_data in data:
#         for key in task_data:
#             words = task_data[key]['text'].split(' ')
#             print(count)
#             count +=1
#             for word in words:
#                 if word not in g_e:
#                     g_e[word] = glove_embeddings.get(word, np.zeros(300, dtype = 'float32'))



# get_glove_dictionary(glove_embeddings)

In [12]:
# import pickle

# with open('glove_embedding.json', 'w') as json_file:
#     json.dump(g_e, json_file)

# with open('fast_text_embedding.json', 'w') as json_file:
#     json.dump(fast_text_embedding, json_file)

# with open('glove_embedding.pkl', 'wb') as pickle_file:
#     pickle.dump(g_e, pickle_file)

# with open('fast_text_embedding.pkl', 'wb') as pickle_file:
#     pickle.dump(fast_text_embedding, pickle_file)


In [4]:

class GRUModel(nn.Module):
    def __init__(self, embedding_dim, output_size):
        super(GRUModel, self).__init__()
        # self.embedding_layer = nn.Embedding.from_pretrained(pretrained_embedding, freeze=True)
        self.gru = nn.GRU(embedding_dim, 128, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        # x = self.embedding_layer(x)
        out, _ = self.gru(x)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

class Task_data(Dataset):
    def __init__(self, data, bio_index, embedding_type):
        self.data = data
        self.length = len(self.data)
        self.bio_index =  bio_index
        self.embedding_type = embedding_type

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        input_sequence = self.data[str(index)]['text'].split(' ')
        sentence_embeddings = []
        if self.embedding_type == "glove":
            sentence_embeddings = [glove_embeddings.get(word, np.zeros(300, dtype = 'float32')) for word in input_sequence]
        elif self.embedding_type == "word2vec":
            sentence_embeddings = [word2vec_u_[word_to_index.get(word, word_to_index['<unk>'])] for word in input_sequence]
        elif self.embedding_type == "fast_text":
            sentence_embeddings = [np.array(fast_text_embedding.get(word, np.zeros(300, dtype = 'float32')), dtype = 'float32') for word in input_sequence]

        sentence_embeddings = np.array(sentence_embeddings, dtype='float32')
        output_sequence = self.data[str(index)]['labels']
        output_labels = [self.bio_index[word] for word in output_sequence]

        # output_labels = np.array(output_labels, dtype='float32')
        return torch.tensor(sentence_embeddings), torch.tensor(output_labels)

In [5]:
def train_model(task, embedding_type, model, optimizer, criterion, device,  num_epochs = 20, batch_size = 1):
    train_dataloader  = None
    val_dataloader  = None
    test_dataloader = None

    if task == 1:
        train_dataloader = DataLoader(Task_data(task1_train_data, bio_mapping_task1, embedding_type), batch_size=batch_size, shuffle=True)
        val_dataloader =  DataLoader(Task_data(task1_val_data, bio_mapping_task1, embedding_type), batch_size=batch_size, shuffle=True)
    elif task == 2:
        train_dataloader = DataLoader(Task_data(task2_train_data, bio_mapping_task2, embedding_type), batch_size=batch_size, shuffle=True)
        val_dataloader =  DataLoader(Task_data(task2_val_data, bio_mapping_task2, embedding_type), batch_size=batch_size, shuffle=True)


    train_losses = []
    val_losses = []
    train_f1_scores = []
    val_f1_scores = []

    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        total_train_loss = 0
        all_train_predictions = []
        all_train_targets = []

        for batch_idx, (inputs, targets) in enumerate(train_dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            
            loss = 0
            for i in range(outputs.size(1)):  # Iterate over time steps
                loss += criterion(outputs[:, i, :], targets[:, i])  # Apply CrossEntropyLoss at each time step
            
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

            all_train_predictions.extend(outputs.argmax(dim=2).view(-1).cpu().numpy())
            all_train_targets.extend(targets.view(-1).cpu().numpy())


        avg_train_loss = total_train_loss / len(train_dataloader)
        train_losses.append(avg_train_loss)

        train_macro_f1 = f1_score(all_train_targets, all_train_predictions, average='macro')
        train_f1_scores.append(train_macro_f1)
        print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss}, Training Macro F1-Score: {train_macro_f1}")

        model.eval()  # Set the model to evaluation mode
        total_val_loss = 0
        all_val_predictions = []
        all_val_targets = []

        with torch.no_grad():
            for val_inputs, val_targets in val_dataloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = model(val_inputs)

                loss = 0
                for i in range(val_outputs.size(1)):  # Iterate over time steps
                    loss += criterion(val_outputs[:, i, :], val_targets[:, i])  

                total_val_loss += loss.item()

                all_val_predictions.extend(val_outputs.argmax(dim=2).view(-1).cpu().numpy())
                all_val_targets.extend(val_targets.view(-1).cpu().numpy())

            avg_val_loss = total_val_loss / len(val_dataloader)
            val_losses.append(avg_val_loss)

            val_macro_f1 = f1_score(all_val_targets, all_val_predictions, average='macro')
            val_f1_scores.append(val_macro_f1)
        print(f"Epoch {epoch + 1},  Validation Loss: {avg_val_loss}, Validation Macro F1-Score: {val_macro_f1}")

    return train_losses, train_f1_scores, val_losses, val_f1_scores

def test_model(task, embedding_type, model, criterion, device, batch_size = 1):
    test_dataloader = None
    if task == 1:
        test_dataloader =  DataLoader(Task_data(task1_test_data, bio_mapping_task1, embedding_type), batch_size=batch_size, shuffle=False)

    elif task == 2:
        test_dataloader =  DataLoader(Task_data(task2_test_data, bio_mapping_task2, embedding_type), batch_size=batch_size, shuffle=False)

    total_test_loss = 0
    all_test_predictions = []
    all_test_targets = []

    with torch.no_grad():
        for test_inputs, test_targets in test_dataloader:
            test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)
            test_outputs = model(test_inputs)

            loss = 0
            for i in range(test_outputs.size(1)):  # Iterate over time steps
                loss += criterion(test_outputs[:, i, :], test_targets[:, i])  

            total_test_loss += loss.item()

            all_test_predictions.extend(test_outputs.argmax(dim=2).view(-1).cpu().numpy())
            all_test_targets.extend(test_targets.view(-1).cpu().numpy())

        avg_test_loss = total_test_loss / len(test_dataloader)
        test_macro_f1 = f1_score(all_test_targets, all_test_predictions, average='macro')
    print(f'Test Loss: {avg_test_loss}, Test Macro F1-Score: {test_macro_f1}')

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = GRUModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

train_losses, train_f1_scores, val_losses, val_f1_scores = train_model(task = 1, embedding_type = "fast_text", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 1, embedding_type = "fast_text", model = model, criterion=criterion, device=device)

torch.save(model, 'task1_gru_fast_text.pth')


plt.plot(range(1, 21), train_losses, label='Training Loss')
plt.plot(range(1, 21), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(range(1, 21), train_f1_scores, label='Training F1')
plt.plot(range(1, 21), val_f1_scores, label='Validation F1')
plt.xlabel('Epochs')
plt.ylabel('F1-score')
plt.legend()
plt.show()

Epoch 1, Training Loss: 19.700520411247393, Training Macro F1-Score: 0.06992336167689264
Epoch 1,  Validation Loss: 15.687431168732406, Validation Macro F1-Score: 0.10953180365168783
Epoch 2, Training Loss: 13.618438193150363, Training Macro F1-Score: 0.1957382735352082
Epoch 2,  Validation Loss: 12.427649989538164, Validation Macro F1-Score: 0.24224223289672153


KeyboardInterrupt: 

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = GRUModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_losses, train_f1_scores, val_losses, val_f1_scores =  train_model(task = 1, embedding_type = "glove", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 1, embedding_type = "glove", model = model, criterion=criterion, device=device)

torch.save(model, 'task1_gru_glove.pth')


plt.plot(range(1, 21), train_losses, label='Training Loss')
plt.plot(range(1, 21), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(range(1, 21), train_f1_scores, label='Training F1')
plt.plot(range(1, 21), val_f1_scores, label='Validation F1')
plt.xlabel('Epochs')
plt.ylabel('F1-score')
plt.legend()
plt.show()

/Users/nalishjain/Acad Sem 6/NLP-Assignments/A2_16/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1, Training Loss: 14.308360046556528, Training Macro F1-Score: 0.24288459091716522
Epoch 1,  Validation Loss: 10.692725662630194, Validation Macro F1-Score: 0.3375941683413264
Epoch 2, Training Loss: 9.100810125831567, Training Macro F1-Score: 0.3800231505050317
Epoch 2,  Validation Loss: 9.209327659466275, Validation Macro F1-Score: 0.38287074255812675
Epoch 3, Training Loss: 7.89741698994375, Training Macro F1-Score: 0.42529003317583125
Epoch 3,  Validation Loss: 8.498299865186732, Validation Macro F1-Score: 0.43741680783221487
Epoch 4, Training Loss: 7.141109836983148, Training Macro F1-Score: 0.4866818735364276
Epoch 4,  Validation Loss: 8.247500391331553, Validation Macro F1-Score: 0.49547481039545
Epoch 5, Training Loss: 6.578506000426399, Training Macro F1-Score: 0.5377460527929239
Epoch 5,  Validation Loss: 7.988114935474785, Validation Macro F1-Score: 0.514030979750835


KeyboardInterrupt: 

In [15]:
model = GRUModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_losses, train_f1_scores, val_losses, val_f1_scores =  train_model(task = 1, embedding_type = "word2vec", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 1, embedding_type = "word2vec", model = model, criterion=criterion, device=device)

torch.save(model, 'task1_gru_word2vec.pth')


plt.plot(range(1, 21), train_losses, label='Training Loss')
plt.plot(range(1, 21), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(range(1, 21), train_f1_scores, label='Training F1')
plt.plot(range(1, 21), val_f1_scores, label='Validation F1')
plt.xlabel('Epochs')
plt.ylabel('F1-score')
plt.legend()
plt.show()

Epoch 1, Training Loss: 17.173340203608305, Training Macro F1-Score: 0.14090418053956702
Epoch 1,  Validation Loss: 13.15755694685806, Validation Macro F1-Score: 0.233369196667654
Epoch 2, Training Loss: 11.625803931502412, Training Macro F1-Score: 0.2967880392883056
Epoch 2,  Validation Loss: 11.146492126649925, Validation Macro F1-Score: 0.326988325008588
Epoch 3, Training Loss: 10.21729689323484, Training Macro F1-Score: 0.3579348298148151
Epoch 3,  Validation Loss: 10.308145308875995, Validation Macro F1-Score: 0.3621748350067143


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = GRUModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_losses, train_f1_scores, val_losses, val_f1_scores = train_model(task = 2, embedding_type = "fast_text", model = model, optimizer=optimizer, criterion=criterion, device=device)

test_model(task = 2, embedding_type = "fast_text", model = model, criterion=criterion, device=device)
torch.save(model, 'gru_2_fast_text.pth')

plt.plot(range(1, 21), train_losses, label='Training Loss')
plt.plot(range(1, 21), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(range(1, 21), train_f1_scores, label='Training F1')
plt.plot(range(1, 21), val_f1_scores, label='Validation F1')
plt.xlabel('Epochs')
plt.ylabel('F1-score')
plt.legend()
plt.show()

In [ ]:
model = GRUModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_losses, train_f1_scores, val_losses, val_f1_scores = train_model(task = 2, embedding_type = "glove", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 2, embedding_type = "glove", model = model, criterion=criterion, device=device)
torch.save(model, 'gru_2_glove.pth')


plt.plot(range(1, 21), train_losses, label='Training Loss')
plt.plot(range(1, 21), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(range(1, 21), train_f1_scores, label='Training F1')
plt.plot(range(1, 21), val_f1_scores, label='Validation F1')
plt.xlabel('Epochs')
plt.ylabel('F1-score')
plt.legend()
plt.show()

In [ ]:
model = GRUModel(300, 27).to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
train_losses, train_f1_scores, val_losses, val_f1_scores = train_model(task = 2, embedding_type = "word2vec", model = model, optimizer=optimizer, criterion=criterion, device=device)
test_model(task = 2, embedding_type = "word2vec", model = model, criterion=criterion, device=device)
torch.save(model, 'gru_2_word2vec.pth')

plt.plot(range(1, 21), train_losses, label='Training Loss')
plt.plot(range(1, 21), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(range(1, 21), train_f1_scores, label='Training F1')
plt.plot(range(1, 21), val_f1_scores, label='Validation F1')
plt.xlabel('Epochs')
plt.ylabel('F1-score')
plt.legend()
plt.show()